# Lab 6

In [1]:
import numpy as np
import pandas as pd 
import seaborn as sns 
import matplotlib as mp
import matplotlib.pyplot as plt 

from scipy import stats
from textwrap import wrap

%matplotlib inline
%config InlineBackend.figure_format = 'pdf'

## Import data

In [2]:
KS = pd.read_excel('/Users/kev/Downloads/BCMB 301A Lab Techniques/Lab 6 - Study of reversible inhibition using beta-galactosidase/Data/kevin.xlsx',
                  header=1)

DI = pd.read_excel('/Users/kev/Downloads/BCMB 301A Lab Techniques/Lab 6 - Study of reversible inhibition using beta-galactosidase/Data/Daniel.xlsx',
                  header=13, index_col=1)

TC = pd.read_excel('/Users/kev/Downloads/BCMB 301A Lab Techniques/Lab 6 - Study of reversible inhibition using beta-galactosidase/Data/Tyler H.xlsx',
                  header=1)

## Clean up data

In [3]:
def abs_ave(D, x0=1, xf=9, dx=10, y0=0, yf=4, n=9, Nanx=[], Nany=[]):
    # 2D data size
    x = xf-x0
    y = yf-y0
    
    # Turn into 3d array
    # Axis 1 = Substrate
    # Axis 2 = Inhibitor
    # Axis 3 = Time
    A = np.zeros((x, y, n))
    for i in range(0,n):
        a = np.array(D.iloc[x0+dx*i:xf+dx*i, y0:yf])
        A[:,:,i] = a 
    
    # Find change in absorbance with time
    Diff = np.diff(A, axis=2)
    Ave = np.nanmean(Diff, axis=2)
    Ave = pd.DataFrame(Ave)
    
    # Get rid of errors
    if len(Nanx)<1:
        for i in range(0, len(Nanx)):
            Ave.iloc[Nanx[i], Nany[i]] = np.nan

    else: 
        Ave += 0
        
    # Get averages of change with respect to time
    Abs = np.zeros((int(x/2), y))
    for i in range(0, int(x/2)):
        Ave_abs = np.nanmean(Ave.iloc[2*i:2*i+1,:], axis=0)
        Abs[i,:] = Ave_abs

    # Label columns
    Abs = pd.DataFrame(Abs)
    Abs.columns = [r'0 $\mu$l', r'10 $\mu$l', r'20 $\mu$l', r'40 $\mu$l']
    ABS = Abs.T
    ABS.columns = [r'10 $\mu$l ONPG', r'20 $\mu$l ONPG', r'40 $\mu$l ONPG', r'80 $\mu$l ONPG']
    return ABS

In [4]:
KS_abs = abs_ave(D=KS, Nanx = [0, 7], Nany = [0, 0])
TC_abs = abs_ave(D=TC)
DI_abs = abs_ave(D=DI, x0=1, xf=9, y0=1, yf=5, dx=12)

In [5]:
KS_abs

,10 $\mu$l ONPG,20 $\mu$l ONPG,40 $\mu$l ONPG,80 $\mu$l ONPG
0 $\mu$l,0.044625,0.039000,0.04275,0.046250
10 $\mu$l,0.029625,0.035125,0.03150,0.041125
20 $\mu$l,0.026125,0.031125,0.03250,0.034125
40 $\mu$l,0.024750,0.026250,0.02775,0.026875


## Function to generate graph

### Get average absorbances (405 nm)

### Get graph